In [79]:
import pandas as pd
import numpy as np
from datetime import timedelta


In [80]:
pd.options.mode.chained_assignment = None #for not showing warning message 
pd.options.display.float_format = '{:,.2f}'.format #for setting thhe float format to this notebook

In [81]:
data = pd.read_excel(r"D:\Analytics\Undercutting\South region - GT\Data\Distributor Invoices.xlsx")

In [82]:
data.shape

(104281, 17)

In [83]:
data.columns =  data.columns.str.strip()
data.columns

Index(['Account', 'DocumentNo', 'Unnamed: 2', 'Doc..Date', 'Pstng Date',
       'CCAr', 'BusA', 'Typ', 'S', 'DD', 'Amount in DC', 'Local Crcy Amt',
       'Net Due Dt', 'Clearing', 'Text', 'LCurr', 'User Name'],
      dtype='object')

In [84]:
data.dtypes

Account           float64
DocumentNo        float64
Unnamed: 2        float64
Doc..Date          object
Pstng Date         object
CCAr               object
BusA               object
Typ                object
S                 float64
DD                float64
Amount in DC      float64
Local Crcy Amt    float64
Net Due Dt         object
Clearing           object
Text               object
LCurr              object
User Name          object
dtype: object

In [85]:
data["Net Due Dt"]           = pd.to_datetime(data["Net Due Dt"],format='%d.%m.%Y')
data["Clearing"]             = data["Clearing"].fillna("01.04.2020")
data["Clearing"]             = pd.to_datetime(data["Clearing"],format='%d.%m.%Y')
data.drop(data[data["Account"].isna()].index,axis =0,inplace=True)
data["Account"]              = data["Account"].astype(int)

In [86]:
invoicedata              = data[["Account","Net Due Dt","Clearing","Amount in DC"]]
invoicedata              = invoicedata[invoicedata["Net Due Dt"] <= "2020-03-31"]
invoicedata["days_diff"] = (data["Net Due Dt"] - data["Clearing"])/timedelta(days=1)
invoicedata["Default"]   = np.where(invoicedata["days_diff"] < 0, "Yes", "No")

In [87]:
invoicedata.head(10)

,Account,Net Due Dt,Clearing,Amount in DC,days_diff,Default
3,2000001,2020-03-30,2020-03-30,"-697,775.28",0.00,No
4,2000001,2020-03-28,2020-03-28,"-15,167.01",0.00,No
5,2000001,2020-03-28,2020-03-28,"15,167.01",0.00,No
6,2000001,2020-03-26,2020-03-26,"20,310.00",0.00,No
7,2000001,2020-03-26,2020-03-26,"-20,310.00",0.00,No
8,2000001,2020-03-24,2020-03-24,"-228,342.39",0.00,No
9,2000001,2020-03-27,2020-03-23,"1,012,193.00",4.00,No
10,2000001,2020-03-27,2020-03-30,"697,775.28",-3.00,Yes
11,2000001,2020-03-23,2020-03-23,"314,417.72",0.00,No
12,2000001,2020-03-23,2020-03-23,"-1,012,193.00",0.00,No


In [90]:
count_dist              = invoicedata.groupby(["Account"]).agg(Count = ("Account","count")).reset_index()
dist_default            = invoicedata.groupby("Account").Default.apply(lambda x: (x == 'Yes').mean())
dist_default            = pd.DataFrame(dist_default).reset_index()
dist_default["Default"] = dist_default["Default"] * 100
dist_default            = dist_default.merge(count_dist,how = "inner",on="Account")

In [91]:
dist_default

,Account,Default,Count
0,2000001,40.90,868
1,2000002,43.16,526
2,2000003,43.06,576
3,2000004,38.78,1029
4,2000005,41.54,573
...,...,...,...
443,2008024,0.00,10
444,2008025,0.00,6
445,2008085,0.00,2
446,2008166,5.00,20


In [78]:
dist_default.to_excel(r"D:\Analytics\Undercutting\South region - GT\Outfiles\dist_default.xlsx",index = False)

In [65]:
invoicedata.to_excel(r"D:\Analytics\Undercutting\South region - GT\Outfiles\invoicedata.xlsx",index = False)